# Mini Project 2 - Part 2

#### Andrew Geday - 260834063
#### Wassim Wazzi - 260825559
#### Cesar Arnouk - 260847683

In [51]:
import numpy as np
import pandas as pd
import sklearn
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

We append the validation set to the training set to have more data to train on, since no CV is implemented here.

In [78]:
# get training data
X = pd.read_csv("data_A2/fake_news/fake_news_train.csv", sep=",") # training data
X_val = pd.read_csv("data_A2/fake_news/fake_news_val.csv", sep=",") # validation data
X = X.append(X_val, ignore_index=True)

Y_train = X.loc[:, 'label'].to_numpy() # array to label string
X_train = X.iloc[:, 0:-1] # drop label in dataframe

In [80]:
X_train

,text
0,Indian fruit is so important to so many people...
1,"FORT WORTH, Texas — Urú Inc. will hold a confe..."
2,"With three of the four new carriers, the Niger..."
3,Let's start with the classic annual dividend r...
4,Following are some of the major events to have...
...,...
21995,"KABUL - In past several weeks, hectic negotiat..."
21996,1) Where is Wan-Bissaka in the young player sh...
21997,It may seem counterintuitive that Calgary shou...
21998,The CEOs of America's biggest banks were summo...


As in the provided tutorial, let's map raw text to features. The way we will do so is by creating a feature for every word that occurs in alln our dataset, and the feature's value will be the word's frequency:

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.text)
X_train_counts

<22000x152673 sparse matrix of type '<class 'numpy.int64'>'
	with 5664886 stored elements in Compressed Sparse Row format>

For example, if we fetch the number of times the word 'indian' occured:

In [56]:
count_vect.vocabulary_.get(u'indian')

68551

In [57]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf.shape)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(22000, 152673)
(22000, 152673)


In [58]:
# convert Numpy sparse matrix to Pandas DataFrame
import scipy.sparse
X_train_tfidf = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf)

We are now done with mapping the text to featueres. Let's observe what our design matrix looks like:

In [59]:
X_train_tfidf

,0,1,2,3,4,5,6,7,8,9,...,152663,152664,152665,152666,152667,152668,152669,152670,152671,152672
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.180029,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,0.000000,0.005808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21996,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21997,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21998,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We now run logistic regression on our data using scikit-learn tools:

In [60]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=0)

# fit the model with data
logreg.fit(X_train_tfidf, Y_train)

#get training data
X_t = pd.read_csv("data_A2/fake_news/fake_news_test.csv", sep=",") # test data
Y_test = X_t.loc[:, 'label'].to_numpy() # array to label string
X_test = X_t.iloc[:, 0:-1] # drop label from dataframe

# feature extracting chain
X_test_counts = count_vect.transform(X_test.text)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# predict on new data
Yh = logreg.predict(X_test_tfidf)

In [61]:
sum(Yh == Y_test)/len(Yh)

0.716

This was just the basic version of what was asked of us. We get an accuracy of 0.716 for this model.
<br>
Now, let's try to go above and beyond.

### Above and beyond

Apart from count vectors and tf-idf, we try to clean our data as much as possible to avoid noise:

In [81]:
import re

def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
#     string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string

# clean the text in our dataframe
for i in range(0,X_train.shape[0]):
    X_train.iloc[i]['text'] = clean_text(X_train.iloc[i]['text'])

Now we preprocess as we did before:

In [82]:
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.text)

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# convert Numpy sparse matrix to Pandas DataFrame
X_train_tfidf = pd.DataFrame.sparse.from_spmatrix(X_train_tfidf)

We predict as we did on the newly processed data:

In [83]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=0)

# fit the model with data
logreg.fit(X_train_tfidf, Y_train)

#get training data
X_t = pd.read_csv("data_A2/fake_news/fake_news_test.csv", sep=",") # test data
Y_test = X_t.loc[:, 'label'].to_numpy() # array to label string
X_test = X_t.iloc[:, 0:-1] # drop label from dataframe

# feature extracting chain
X_test_counts = count_vect.transform(X_test.text)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# predict on new data
Yh = logreg.predict(X_test_tfidf)

In [84]:
sum(Yh == Y_test)/len(Yh)

0.7043333333333334

In [45]:
from sklearn.linear_model import LogisticRegressionCV

cv_values = [5,10]
accuracies = {}

for cv in cv_values:
    print('Now trying out ' + str(cv) + '-fold cross-validation')
    # instantiate the model (using the default parameters)
    logregCV = LogisticRegressionCV(cv=cv, random_state=0)

    # fit the model with data
    logregCV.fit(X_train_tfidf, Y_train)

    #get training data
    X_t = pd.read_csv("data_A2/fake_news/fake_news_test.csv", sep=",") # test data
    Y_test = X_t.loc[:, 'label'].to_numpy() # array to label string
    X_test = X_t.iloc[:, 0:-1] # drop label from dataframe

    # feature extracting chain
    X_test_counts = count_vect.transform(X_test.text)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)

    # predict on new data
    Yh = logregCV.predict(X_test_tfidf)
    
    accuracies[cv] = sum(Yh == Y_test)/len(Yh)

Now trying out 5-fold cross-validation
Now trying out 10-fold cross-validation


In [46]:
accuracies

{5: 0.7216666666666667, 10: 0.7216666666666667}